In [3]:
import eco2ai
tracker = eco2ai.Tracker(project_name="lct_12_sber_multimodal_recsys", 
                         experiment_description="trx_pyspark_v1_2_train_test", ignore_warnings=True)
tracker.start()

In [4]:
import os
os.environ["OMP_NUM_THREADS"] = "8"#4

import pandas as pd
import numpy as np
import torch
from functools import partial
import pytorch_lightning as pl
import warnings
import pickle
import time
warnings.filterwarnings("ignore")

from torch.utils.data import DataLoader

from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset, ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.preprocessing import PandasDataPreprocessor, PysparkDataPreprocessor
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from ptls.data_load.datasets import ParquetDataset, ParquetFiles
import ptls

from tqdm.auto import tqdm
#import lightgbm as ltb
from datetime import datetime

import multiprocessing
print('cpu', multiprocessing.cpu_count())

import psutil    
print('ram', round(psutil.virtual_memory().total/1024/1024/1024,1), 'gb')  # total physical memory in Bytes

debug=False

print('torch', torch.__version__)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('pytorch-lightning', pl.__version__)
print(time.ctime())

cpu 16
ram 188.7 gb
torch 1.13.1+cu116
pandas 1.4.3
numpy 1.26.0
pytorch-lightning 2.2.5
Sat Jun 15 10:57:06 2024


In [5]:
!nvidia-smi

Sat Jun 15 10:57:06 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:8B:00.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      4MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:8C:00.0 Off |                    0 |
| N/A   

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
import os
print('pyspark', pyspark.__version__)
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1" 
# без этой строчки у нас будет возникать постоянное предупреждение с просьбой установить эту переменную в значение 1, что мы заранее и делаем

pyspark 3.5.0


In [7]:
spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "16g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "16g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "16")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")
spark_conf.set("spark.executor.heartbeatInterval", "200000")
spark_conf.set("spark.network.timeout", "300000")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

24/06/15 10:57:10 WARN Utils: Your hostname, gpupreemnew resolves to a loopback address: 127.0.1.1; using 10.129.0.19 instead (on interface eth0)
24/06/15 10:57:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/15 10:57:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/15 10:57:12 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


[('spark.executor.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.memoryOverhead', '4g'),
 ('spark.app.startTime', '1718449031855'),
 ('spark.local.dir', '../../spark_local_dir'),
 ('spark.cores.max', '

## 1. Предобратока данных

In [ ]:
%%time
transactions_train = spark.read.parquet("Hackathon/trx_train.parquet")
transactions_test = spark.read.parquet("Hackathon/trx_test.parquet")

In [ ]:
print(transactions_train.count(), transactions_test.count())

In [ ]:
%%time
if debug:
    train_clients = [i['client_id'] for i in transactions_train.select('client_id').distinct().collect()]
    print(len(train_clients))
    train_clients = pd.Series(train_clients).sample(random_state=37, frac=0.1).values.tolist()
    print(len(train_clients))
    transactions_train = transactions_train.filter(col('client_id').isin(train_clients))
    #print(transactions_train['client_id'].n_unique())

In [ ]:
%%time
if debug:
    test_clients = [i['client_id'] for i in transactions_test.select('client_id').distinct().collect()]
    print(len(test_clients))
    test_clients = pd.Series(test_clients).sample(random_state=37, frac=0.1).values.tolist()
    print(len(test_clients))
    transactions_test = transactions_test.filter(col('client_id').isin(test_clients))
    #print(transactions_test['client_id'].n_unique())

In [ ]:
# объединяем train и test, чтобы историческая информация о поведении тестовых пользователей тоже использовалась для обучения
transactions_train = reduce(DataFrame.unionAll, [transactions_train, transactions_test])
print(transactions_train.count())

In [ ]:
%%time

preprocessor = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="event_time",
    event_time_transformation="dt_to_timestamp",
    cols_category=["event_type",
                   "event_subtype",
                   "currency",
                   "src_type11",
                   "src_type12",
                   "dst_type11",
                   "dst_type12",
                   "src_type21",
                   "src_type22",
                   "src_type31",
                   "src_type32"],
    cols_identity="amount",
    #return_records=False,
)

processed_train = preprocessor.fit_transform(transactions_train)
processed_test = preprocessor.transform(transactions_test)

In [ ]:
processed_train.dtypes

In [ ]:
with open('preprocessor_pyspark_v1.pickle', 'wb') as f:
    pickle.dump(preprocessor, f)

In [ ]:
%%time
processed_train.write.parquet('processed_train_pyspark_v1.parquet', mode='overwrite')
processed_test.write.parquet('processed_test_pyspark_v1.parquet', mode='overwrite')

In [ ]:
print(processed_train.count(), processed_test.count())
print(len(processed_train.select('client_id').distinct().collect()), len(processed_test.select('client_id').distinct().collect()))
#processed_train.show(5)

In [ ]:
del processed_train, processed_test

## 2. Тренировка модели

In [8]:
with open('preprocessor_pyspark_v1.pickle', 'rb') as f:
    preprocessor = pickle.load(f)

**Обработка датасета:**

- Транзакции, у которых размер < min_seq_len выкидываются
- Транзакции, у которых длина > max_seq_len, обрезаются и конвертируются в torch.tensor
- Не нужные для CoLES фичи удаляются

In [9]:
%%time
train = MemoryMapDataset(
    data=ParquetDataset(ParquetFiles('processed_train_pyspark_v1.parquet')),#processed_train.toPandas().to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch()
    ]
)

CPU times: user 3min 2s, sys: 44.2 s, total: 3min 46s
Wall time: 2min 54s


In [10]:
%%time
test = MemoryMapDataset(
    data=ParquetDataset(ParquetFiles('processed_test_pyspark_v1.parquet')),#processed_test.toPandas().to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=4096),
        ToTorch()
    ]
)

CPU times: user 42.6 s, sys: 11.6 s, total: 54.2 s
Wall time: 40.9 s


In [11]:
%%time
train_ds = ColesDataset(
    data=train,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

valid_ds = ColesDataset(
    data=test,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=180
    )
)

CPU times: user 45 µs, sys: 12 µs, total: 57 µs
Wall time: 62.2 µs


In [12]:
%%time
train_dl = PtlsDataModule(
    train_data=train_ds,
    train_num_workers=16,
    train_batch_size=256,
    valid_data=valid_ds,
    valid_num_workers=16,
    valid_batch_size=256
)

CPU times: user 822 µs, sys: 0 ns, total: 822 µs
Wall time: 832 µs


- numeric_values обрабатываются как BatchNorm+Linear
- embedidngs - nn.Embedidngs

In [13]:
%%time
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount': 'log'},
    embeddings={
        "event_type": {'in': preprocessor.get_category_dictionary_sizes()["event_type"], "out": 24},
        "event_subtype": {'in': preprocessor.get_category_dictionary_sizes()["event_subtype"], "out": 24},
        'src_type11': {'in': preprocessor.get_category_dictionary_sizes()["src_type11"], 'out': 24},
        'src_type12': {'in': preprocessor.get_category_dictionary_sizes()["src_type12"], 'out': 24},
        'dst_type11': {'in': preprocessor.get_category_dictionary_sizes()["dst_type11"], 'out': 24},
        'dst_type12': {'in': preprocessor.get_category_dictionary_sizes()["dst_type12"], 'out': 24},
        'src_type22': {'in': preprocessor.get_category_dictionary_sizes()["src_type22"], 'out': 24},
        'src_type31': {'in': preprocessor.get_category_dictionary_sizes()["src_type31"], 'out': 24},
        'src_type32': {'in': preprocessor.get_category_dictionary_sizes()["src_type32"], 'out': 24},
      }
)

CPU times: user 71 µs, sys: 18 µs, total: 89 µs
Wall time: 95.6 µs


- **TrxEncoder** - обрабатывает каждую тразнакцию (строит для неё эмбеддиг)
- **SeqEncoder** - обрабатывает последовательность

In [14]:
%%time
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=256,
    type='gru',
)

CPU times: user 29.3 ms, sys: 0 ns, total: 29.3 ms
Wall time: 12.5 ms


In [15]:
%%time
model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=3, gamma=0.9025)
)

CPU times: user 31.2 ms, sys: 0 ns, total: 31.2 ms
Wall time: 30.3 ms


In [16]:
%%time
trainer = pl.Trainer(
    max_epochs=45,
    limit_val_batches=5000,
    devices=[0],
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    logger=pl.loggers.TensorBoardLogger(
        save_dir='./logdir',
        name='baseline_pyspark_v1_2_result'
    ),
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=5000, save_top_k=-1),
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


CPU times: user 329 ms, sys: 41 ms, total: 371 ms
Wall time: 377 ms


In [17]:
%%time
trainer.fit(model, train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 451 K 
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
451 K     Trainable params
0         Non-trainable params
451 K     Total params
1.805     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=45` reached.


CPU times: user 6h 14min 12s, sys: 53min 31s, total: 7h 7min 44s
Wall time: 7h 5min 43s


In [18]:
torch.save(model.state_dict(), './model_pyspark_v1_2.pt')

## 3. Inference

Для каждого пользователя известно 12 таргетов, инференс происходит следующим образом:

Чтобы не происходило лика нужно для каждого клиента делать срез до текущего месяца:

Берутся все тразнакции за первый месяц, им соответствует 1-ый таргет из 12,
потом берутся транзакции за первый и второй месяц пользователя и им соотвествует 2-ой таргет и так далее.
То есть для данного пользователя, имеющего транзакции за год, мы можем получить 12 эмбеддингов, каждому из которых соответствует 1 таргет

In [19]:
class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time

    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()

                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()

                year_event_time = datetime(self._year, 1, 1).timestamp()

                mask = features[self._col_time] < month_event_time

                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()
                    elif key != self._col_id:
                        features[key] = tensor[mask]

                features[self._col_id] += '_month=' + str(month)

                yield features

def collate_feature_dict_with_target(batch, col_id='client_id', targets=False):
    batch_ids = []
    target_cols = []
    for sample in batch:
        batch_ids.append(sample[col_id])
        del sample[col_id]

        if targets:
            target_cols.append([sample[f'target_{i}'] for i in range(1, 5)])
            del sample['target_1']
            del sample['target_2']
            del sample['target_3']
            del sample['target_4']

    padded_batch = collate_feature_dict(batch)
    if targets:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids


class InferenceModuleMultimodal(pl.LightningModule):
    def __init__(self, model, pandas_output=True, drop_seq_features=True, model_out_name='out'):
        super().__init__()

        self.model = model
        self.pandas_output = pandas_output
        self.drop_seq_features = drop_seq_features
        self.model_out_name = model_out_name

    def forward(self, x):
        x_len = len(x)
        if x_len == 3:
            x, batch_ids, target_cols = x
        else:
            x, batch_ids = x

        out = self.model(x)
        if x_len == 3:
            target_cols = torch.tensor(target_cols)
            x_out = {
                'client_id': batch_ids,
                'target_1': target_cols[:, 0],
                'target_2': target_cols[:, 1],
                'target_3': target_cols[:, 2],
                'target_4': target_cols[:, 3],
                self.model_out_name: out
            }
        else:
            x_out = {
                'client_id': batch_ids,
                self.model_out_name: out
            }
        torch.cuda.empty_cache()

        if self.pandas_output:
            return self.to_pandas(x_out)
        return x_out

    @staticmethod
    def to_pandas(x):
        expand_cols = []
        scalar_features = {}

        for k, v in x.items():
            if type(v) is torch.Tensor:
                v = v.cpu().numpy()

            if type(v) is list or len(v.shape) == 1:
                scalar_features[k] = v
            elif len(v.shape) == 2:
                expand_cols.append(k)
            else:
                scalar_features[k] = None

        dataframes = [pd.DataFrame(scalar_features)]
        for col in expand_cols:
            v = x[col].cpu().numpy()
            dataframes.append(pd.DataFrame(v, columns=[f'{col}_{i:04d}' for i in range(v.shape[1])]))

        return pd.concat(dataframes, axis=1)

In [20]:
model.load_state_dict(torch.load('model_pyspark_v1_2.pt'))

<All keys matched successfully>

In [21]:
%%time
test = MemoryMapDataset(
    data=ParquetDataset(ParquetFiles('processed_test_pyspark_v1.parquet')),#processed_test.toPandas().to_dict("records"),
    i_filters=[
        ISeqLenLimit(max_seq_len=4096),
        FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        ToTorch(),
    ]
)

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:07:03 UTC)" was missed by 0:00:05.400292


CPU times: user 45.9 s, sys: 13.2 s, total: 59.1 s
Wall time: 44.9 s


In [22]:
inference_test_dl = DataLoader(
        dataset=test,
        collate_fn=collate_feature_dict_with_target,
        shuffle=False,
        num_workers=0,
        batch_size=256,
    )

In [23]:
inf_module = InferenceModuleMultimodal(
        model=model,
        pandas_output=True,
        drop_seq_features=True,
        model_out_name='emb',
    )

In [24]:
trainer = pl.Trainer(devices=[0], max_epochs=-1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [25]:
inf_test_embeddings = pd.concat(
        trainer.predict(inf_module, inference_test_dl)
    )
inf_test_embeddings.to_parquet("test_pyspark_v1_2.parquet", index=False, engine="pyarrow", compression="snappy")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [26]:
print(inf_test_embeddings.shape)
inf_test_embeddings.head(2)

(225744, 257)


,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,0003304a0f65d675ddfbc0691e0c564d26a4c9e08edf67...,0.991176,-0.775125,0.757665,0.859406,0.418732,-0.995711,0.998106,0.843568,-0.979261,...,0.945397,1.000000,0.911293,-0.924687,-0.142008,-0.616675,-0.753703,0.148448,0.989438,-0.536777
1,000bd0936be1275d33846423883b01b78b1c38f992d043...,0.975135,-0.174958,0.624342,0.599224,-0.862071,-0.991283,0.981792,0.824762,-0.981896,...,0.949109,0.899744,0.939540,-0.554023,0.175411,-0.687313,-0.849484,0.620372,0.955219,-0.298517


Файл **sample_submission** составляется из **client_id** файла **test_target_b**. Так как не у всех пользователей может быть транзакционная история, мы для простоты заполняем их фичи нулями.

In [27]:
%%time
test_target_b = pd.read_parquet("Hackathon/test_target_b.parquet")

CPU times: user 682 ms, sys: 257 ms, total: 938 ms
Wall time: 442 ms


In [28]:
not_only_trx = pd.DataFrame({"client_id": test_target_b["client_id"].unique()}).merge(inf_test_embeddings, how="left").fillna(0)
not_only_trx

,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671...,0.993582,0.481427,0.360624,0.360467,-0.897656,-0.995617,0.999655,0.900769,-0.985567,...,0.952166,0.998346,0.914159,-0.749460,0.480764,-0.703740,-0.865454,0.163078,0.995917,-0.308854
1,0433d23e224b7a520656da6181efadb8d556bb293158c9...,0.925823,-0.232634,0.547064,0.287713,-0.858418,-0.939000,0.875962,0.781385,-0.946616,...,0.933649,0.937286,0.864485,-0.285547,0.085621,-0.628492,-0.771841,0.540046,0.846965,-0.257493
2,f2ce8b292e5f9f778f3e20db7608ac76dc8812113a2631...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,4f807e8b163c653bcaeff9f925983568f4c3e6b1a1f231...,0.990614,0.196399,0.549216,0.756582,-0.874799,-0.995653,0.998794,0.867110,-0.984767,...,0.953464,0.997684,0.924558,-0.716014,0.243287,-0.700432,-0.855004,0.455324,0.989093,-0.305245
4,64369f6f8ae1b719332ee1bfb2b454e642b2053d2c9b8a...,0.979018,-0.325149,0.691433,0.794108,-0.875588,-0.992033,0.987105,0.828518,-0.983828,...,0.953093,0.996554,0.915376,-0.739537,0.057866,-0.690133,-0.843768,0.681804,0.966470,-0.282144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140483,d49a66825bb16ceb5b6a01126e1f2391b085dba8da44ee...,0.991935,-0.443565,0.615571,0.921626,-0.862225,-0.995579,0.998275,0.898744,-0.984747,...,0.951937,0.977213,0.944685,-0.918739,0.161215,-0.703765,-0.869006,0.772829,0.997627,-0.303569
140484,f772af6720c0b591d49b97946c5e420c1c077affc0f7c7...,0.974042,-0.147920,0.628482,0.548956,-0.846709,-0.992496,0.988769,0.823263,-0.976343,...,0.951666,0.984762,0.933384,-0.505999,0.154068,-0.691299,-0.841191,0.597951,0.957712,-0.298026
140485,06b282335bc4853f888e1ab50a6ba23a8e420d42313959...,0.987940,-0.337652,0.695684,0.738949,-0.859841,-0.995145,0.995775,0.861593,-0.984743,...,0.951058,0.966477,0.945906,-0.670607,0.079111,-0.702650,-0.855105,0.682940,0.984890,-0.302003
140486,90d423a25d7cdaf674f7d78bc37d88830443ff17717e02...,0.994131,-0.199934,0.713510,0.884460,-0.880511,-0.995466,0.999745,0.896407,-0.985092,...,0.952036,0.996287,0.933817,-0.933032,0.530462,-0.717942,-0.859598,0.622882,0.997916,-0.305930


In [29]:
not_only_trx.to_parquet("test_pyspark_v1_2.parquet", index=False, engine="pyarrow", compression="snappy")

In [30]:
print(not_only_trx.shape)
not_only_trx.head(2)

(140488, 257)


,client_id,emb_0000,emb_0001,emb_0002,emb_0003,emb_0004,emb_0005,emb_0006,emb_0007,emb_0008,...,emb_0246,emb_0247,emb_0248,emb_0249,emb_0250,emb_0251,emb_0252,emb_0253,emb_0254,emb_0255
0,2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671...,0.993582,0.481427,0.360624,0.360467,-0.897656,-0.995617,0.999655,0.900769,-0.985567,...,0.952166,0.998346,0.914159,-0.749460,0.480764,-0.703740,-0.865454,0.163078,0.995917,-0.308854
1,0433d23e224b7a520656da6181efadb8d556bb293158c9...,0.925823,-0.232634,0.547064,0.287713,-0.858418,-0.939000,0.875962,0.781385,-0.946616,...,0.933649,0.937286,0.864485,-0.285547,0.085621,-0.628492,-0.771841,0.540046,0.846965,-0.257493


In [31]:
not_only_trx.client_id.values[0]

'2b7ff0c1c99cefe259ed83c5dfa0a403f2cbc88032b671d0bcb425e90932db07'

In [32]:
del inf_test_embeddings

In [33]:
for j in [1, 3, 5, 7, 9, 11]:
    print('='*80)
    print(j)
    print(time.ctime())

    train = MemoryMapDataset(
        #data=processed_train.merge(processed_target9.drop(["event_time", 'target_len','new_mon'], axis=1), 
        #                           on="client_id", how="inner").to_dict("records"),
        ParquetDataset(ParquetFiles('processed_train_pyspark_v1.parquet')),
        
        i_filters=[
            ISeqLenLimit(max_seq_len=4096),
            FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),#
            GetSplit(start_month=j, end_month=j+1),
            ToTorch(),
        ]
    )

    inference_train_dl = DataLoader(
        dataset=train,
        #collate_fn=partial(collate_feature_dict_with_target, targets=True),
    collate_fn=collate_feature_dict_with_target,
        shuffle=False,
        num_workers=0,
        batch_size=256,
    )

    inf_train_embeddings = pd.concat(trainer.predict(inf_module, inference_train_dl))

    inf_train_embeddings.to_parquet("train_pyspark_v1_2_m_"+str(j)+'_'+str(j+1)+".parquet", index=False, engine="pyarrow", compression="snappy")
    print(inf_train_embeddings.shape)
    del inf_train_embeddings, train

1
Sat Jun 15 18:09:29 2024


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:11:03 UTC)" was missed by 0:00:06.441183
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:14:43 UTC)" was missed by 0:00:08.149081
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:19:33 UTC)" was missed by 0:00:07.136929
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |                                             | 0/? [00:00<?, ?it/s]

(2162742, 257)
3
Sat Jun 15 18:29:42 2024


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:32:23 UTC)" was missed by 0:00:03.444165
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:39:53 UTC)" was missed by 0:00:05.030341
Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 18:44:33 UTC)" was missed by 0:00:09.600140
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

(2162742, 257)
11
Sat Jun 15 20:39:43 2024


Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 20:52:33 UTC)" was missed by 0:00:03.901395
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |                                             | 0/? [00:02<?, ?it/s]

Run time of job "Tracker._func_for_sched (trigger: interval[0:00:10], next run at: 2024-06-15 21:00:13 UTC)" was missed by 0:00:01.807374


(2162742, 257)


In [34]:
tracker.stop()

In [ ]:
time.sleep(30)

In [ ]:
#!sudo poweroff